# Data Import, Cleaning, and EDA for AirBnB listings in Mexico City
from insideairbnb.com

## Import Libraries

In [1]:
import pandas as pd
import numpy as np

## Import Data
Free Data is available quarterly so I have imported listings one year back starting in June 2022

In [2]:
june = pd.read_csv('./data/listings_june.csv')
june['last_scraped'].unique()

array(['2022-06-21', '2022-06-22'], dtype=object)

In [3]:
march = pd.read_csv('./data/listings_march.csv')
march['last_scraped'].unique()

array(['2022-03-27', '2022-03-26'], dtype=object)

In [4]:
dec = pd.read_csv('./data/listings_dec.csv')
dec['last_scraped'].unique()

array(['2021-12-26', '2021-12-25', '2021-12-29'], dtype=object)

In [5]:
sep = pd.read_csv('./data/listings_sept.csv')
sep['last_scraped'].unique()

array(['2021-09-30', '2021-09-29', '2021-09-28', '2021-10-06'],
      dtype=object)

## Drop features with high proportion of missing values

In [6]:
# Determine NaN rates and drop features with > 80% NaNs
thresh = 0.95
nan_rates = june.isna().sum(axis=0).sort_values(ascending=False) / float(
    len(june)
)
june.drop(nan_rates[nan_rates > thresh].index, axis=1, inplace=True)
print(
    f"Removed {len(nan_rates[nan_rates > thresh])} features with > {thresh*100}% missing values"
)

# Also drop rows with all NaNs
june.dropna(how="all", inplace=True)

Removed 4 features with > 95.0% missing values


In [7]:
june.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21669 entries, 0 to 21668
Data columns (total 70 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            21669 non-null  int64  
 1   listing_url                                   21669 non-null  object 
 2   scrape_id                                     21669 non-null  int64  
 3   last_scraped                                  21669 non-null  object 
 4   name                                          21666 non-null  object 
 5   description                                   20624 non-null  object 
 6   neighborhood_overview                         13922 non-null  object 
 7   picture_url                                   21669 non-null  object 
 8   host_id                                       21669 non-null  int64  
 9   host_url                                      21669 non-null 

In [8]:
june['host_response_time'].unique()

array([nan, 'a few days or more', 'within an hour', 'within a few hours',
       'within a day'], dtype=object)

In [9]:
# Must revisit this drop since there is useful info here that I'm dropping for simplicity
june.drop(columns=['neighbourhood', 'property_type', 'amenities', 'calendar_last_scraped'], inplace=True)

In [10]:
june = june.astype(
    {
        "last_scraped": "datetime64",
        "host_since": "datetime64",
        'first_review': 'datetime64',
        'last_review': 'datetime64',
    }
)

In [11]:
june["host_response_rate"] = (
    june["host_response_rate"].str.replace("%", "").fillna(0).astype("float") / 100.0
)
june["host_response_rate"].unique

<bound method Series.unique of 0        0.0
1        0.0
2        0.0
3        1.0
4        1.0
        ... 
21664    1.0
21665    1.0
21666    0.5
21667    1.0
21668    1.0
Name: host_response_rate, Length: 21669, dtype: float64>

In [12]:
june["host_acceptance_rate"] = (
    june["host_acceptance_rate"].str.replace("%", "").fillna(0).astype("float") / 100.0
)
june["host_acceptance_rate"].unique

<bound method Series.unique of 0        0.00
1        0.00
2        0.00
3        0.66
4        1.00
         ... 
21664    1.00
21665    1.00
21666    0.00
21667    1.00
21668    1.00
Name: host_acceptance_rate, Length: 21669, dtype: float64>

In [13]:
june["host_is_superhost"] = (
    june["host_is_superhost"].map({"f": 0, "t": 1}).astype("bool")
)

june["host_is_superhost"].unique

<bound method Series.unique of 0        False
1        False
2        False
3        False
4         True
         ...  
21664    False
21665    False
21666    False
21667    False
21668     True
Name: host_is_superhost, Length: 21669, dtype: bool>

In [14]:
june["host_identity_verified"] = (
    june["host_identity_verified"].map({"f": 0, "t": 1}).astype("bool")
)

june["host_identity_verified"].unique()

array([ True, False])

In [15]:
categoricals = ['room_type', 'neighbourhood_cleansed', 'host_response_time']

In [16]:
june['bathrooms_text'] = june["bathrooms_text"].map(
    {
        "3 baths": "3 a",
        "1 bath": "1 a",
        "1 private bath": "1 a",
        "5.5 baths": "5.5 a",
        "1 shared bath": "1 b",
        "1.5 baths": "1.5 a",
        "1.5 shared baths": "1.5 b",
        "2 baths": "2 a",
        "5 baths": "5 a",
        "4.5 baths": "4.5 a",
        "2.5 baths": "2.5 a",
        "3.5 baths": "3.5 a",
        "2 shared baths": "2 b",
        "5.5 shared baths": "5.5 b",
        "6 baths": "6 b",
        "3 shared baths": "3 b",
        "8 shared baths": "8 b",
        "2.5 shared baths": "2.5 b",
        "Half-bath": "0.5 a",
        "3.5 shared baths": "3.5 b",
        "4 baths": "4 a",
        "10 baths": "10 a",
        "5 shared baths": "5 b",
        "16 baths": "16 a",
        "4 shared baths": "4 a",
        "0 baths": "0 a",
        "7 baths": "7 a",
        "20 baths": "20 a",
        "4.5 shared baths": "4.5 b",
        "0 shared baths": "0 b",
        "50 baths": "50 a",
        "10 shared baths": "10 b",
        "Shared half-bath": "0.5 b",
        "6.5 shared baths": "6.5 b",
        "8.5 baths": "8.5 a",
        "7.5 baths": "7.5 a",
        "9 baths": "9 a",
        "11.5 baths": "11.5 a",
        "8 baths": "8 a",
        "Private half-bath": "0.5 a",
        "14 baths": "14 a",
        "11 baths": "11 a",
        "6.5 baths": "6.5 a",
        "6 shared baths": "6 b",
        "8.5 shared baths": "8.5 b",
        "13 baths": "13 a",
        "15 shared baths": "15 b",
        "17 baths": "17 a",
        "12 shared baths": "12 b",
        "7.5 shared baths": "7.5 b",
        "7 shared baths": "7 b",
        "9.5 baths": "9.5 a",
        "30 baths": "30 a",
        "26 baths": "26 a",
    }
)

In [17]:
june[['bathrooms_count', 'bathrooms_type']] = june['bathrooms_text'].str.split(expand=True)

In [18]:
june['price'] = june['price'].str.replace('[\$,]', '', regex=True).astype('float')

In [19]:
june["instant_bookable"] = (
    june["instant_bookable"].map({"f": 0, "t": 1}).astype("bool")
)

june["instant_bookable"].unique()

array([ True, False])

In [20]:
june["has_availability"] = (
    june["has_availability"].map({"f": 0, "t": 1}).astype("bool")
)

june["has_availability"].unique()

array([ True, False])

In [21]:
june = pd.get_dummies(june, columns=categoricals)

In [22]:
june.corr()['price'].sort_values().head()

room_type_Private room                       -0.052438
neighbourhood_cleansed_Benito Juárez         -0.020908
longitude                                    -0.020074
neighbourhood_cleansed_Coyoacán              -0.017197
neighbourhood_cleansed_Venustiano Carranza   -0.014896
Name: price, dtype: float64

In [25]:
june.corr()['price'].sort_values(ascending=False).head(6)

price                                    1.000000
accommodates                             0.103180
bedrooms                                 0.080340
beds                                     0.076775
room_type_Entire home/apt                0.053347
neighbourhood_cleansed_Miguel Hidalgo    0.039532
Name: price, dtype: float64

In [26]:
june.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21669 entries, 0 to 21668
Data columns (total 89 columns):
 #   Column                                         Non-Null Count  Dtype         
---  ------                                         --------------  -----         
 0   id                                             21669 non-null  int64         
 1   listing_url                                    21669 non-null  object        
 2   scrape_id                                      21669 non-null  int64         
 3   last_scraped                                   21669 non-null  datetime64[ns]
 4   name                                           21666 non-null  object        
 5   description                                    20624 non-null  object        
 6   neighborhood_overview                          13922 non-null  object        
 7   picture_url                                    21669 non-null  object        
 8   host_id                                        21669 non